In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# Carga de Dados e Estatísticas Descritivas

print("--- 1. Carregando Dados e Estatísticas ---")
df = pd.read_csv('StudentsPerformance.csv')

print("Informações da Base:")
print(df.info())
print("\nEstatísticas Descritivas:")
print(df.describe(include='all'))

# Transformações (Limpeza e Feature Engineering)

print("\n--- 2. Aplicando Transformações ---")

# Remover duplicatas
initial_rows = df.shape[0]
df = df.drop_duplicates()
print(f"Linhas removidas (duplicadas): {initial_rows - df.shape[0]}")

# Criar Target e Codificar Variáveis
# Definimos o Target: 1 se 'math score' >= 60 (Aprovado), senão 0 (Reprovado)
df['target'] = df['math score'].apply(lambda x: 1 if x >= 60 else 0)
print(f"\nTarget criado (Aprovado em Math): \n{df['target'].value_counts()}")

# Codificação de variáveis categóricas (Texto -> Número)
cat_columns = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
le = LabelEncoder()

for col in cat_columns:
    # Cria colunas novas com sufixo '_code'
    df[col + '_code'] = le.fit_transform(df[col])

print("\nVariáveis categóricas codificadas com sucesso.")


# Divisão em Treino, Validação e Teste

print("\n--- 3. Dividindo os Conjuntos ---")

# Features: Dados demográficos codificados + notas de leitura/escrita
feature_cols = [c + '_code' for c in cat_columns] + ['reading score', 'writing score']
X = df[feature_cols]
y = df['target'] # O target foi derivado da 'math score', então não usamos ela no X

# Divisão: 70% Treino, 15% Validação, 15% Teste
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Tamanho Treino: {X_train.shape[0]}")
print(f"Tamanho Validação: {X_val.shape[0]}")
print(f"Tamanho Teste: {X_test.shape[0]}")


# Treinamento e Avaliação do Modelo

print("\n--- 4. Treinando o Modelo (Random Forest) ---")

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliação
y_pred = model.predict(X_test)

print("\n--- 5. Resultados Finais (Base de Teste) ---")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

acc_test = accuracy_score(y_test, y_pred)
print(f"\nAcurácia no Teste: {acc_test:.4f}")

# Predição de Exemplo

print("\n--- 6. Exemplo de Predição ---")
sample_idx = 5
sample_data = X_test.iloc[[sample_idx]]
real_value = y_test.iloc[sample_idx]
pred_value = model.predict(sample_data)[0]

print(f"Dados do Aluno: \n{sample_data.to_dict(orient='records')[0]}")
print(f"Aprovado em Matemática? (Real): {'Sim' if real_value==1 else 'Não'}")
print(f"Predição do Modelo: {'Sim' if pred_value==1 else 'Não'}")

--- 1. Carregando Dados e Estatísticas ---
Informações da Base:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB
None

Estatísticas Descritivas:
        gender race/ethnicity parental level of education     lunch  \
count     1000           1000                        1000      1000   
unique       2              5       